# Dask Bag

Материалы: 
* Макрушин С.В. Лекция 12: Map-Reduce
* https://docs.dask.org/en/latest/bag.html
* JESSE C. DANIEL. Data Science with Python and Dask. 

In [35]:
import dask.bag as db
import json
import re


## Задачи для совместного разбора

1. Считайте файл `Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt` и разбейте на предложения. Подсчитайте длину (в кол-ве символов) каждого предложения.

In [29]:
path = "12_dask_bag_data/Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt"
text_bag = db.read_text(path, encoding='windows-1251')

def split_into_sentences(text):
    sentences = re.split(r'(?<=[.!?])\s+(?=[А-ЯA-Z])', text)
    return [s.strip() for s in sentences if s.strip()]

sentences = text_bag.map(split_into_sentences).flatten()
sentences = sentences.filter(lambda s: len(s) > 0)

sentence_lengths = sentences.map(len)

sentence_lengths.take(10)


(98, 73, 93, 17, 28, 5, 30, 7, 52, 41)

2. Считайте файл `Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt` и разбейте на предложения. Выведите предложения, длина которых не более 10 символов.

In [31]:

short_sentences = sentences.filter(lambda s: len(s) <= 10)

short_sentences.take(20)  


('Игрок',
 'Глава I',
 'Глава II',
 'Глава III',
 '—\xa0Как!',
 'Глава IV',
 'Глава V',
 'Чудо!',
 'Как зачем?',
 '—\xa0Кого?',
 '—\xa0Зачем?',
 'Глава VI',
 '—\xa0Никаких.',
 'Вот и все.',
 'Прощайте.',
 'Понимаю-с.',
 'Глава VII',
 'Ваша П.',
 'Р.',
 'S.')

3. На основе списка предложений из задачи 1-2 создайте `dask.bag`. Рассчитайте среднюю длину предложений в тексте.

In [33]:
total_chars = sentences.map(len).sum()
total_sentences = sentences.count()


avg_length_alt = sentences.map(len).mean()
avg_length_alt.compute()

79.8760539629005

4. На основе файла `addres_book.json` создайте `dask.bag`. Посчитайте количество мобильных и рабочих телефонов в наборе данных

In [57]:
address_path = "12_dask_bag_data/addres-book.json"

with open(address_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

address_bag = db.from_sequence(data)

def classify_phones_simple(person):
    mobile = 0
    work = 0
    
    for phone in person.get('phones', []):
        phone_number = phone.get('phone', '')
        
        # Мобильные: начинаются с +7, 8, или содержат (916) и т.д.
        if (phone_number.startswith('+7') or 
            phone_number.startswith('8') or
            re.search(r'\(9\d{2}\)', phone_number)):
            mobile += 1
        else:
            work += 1
    
    return (mobile, work)

phone_counts = address_bag.map(classify_phones_simple)
total_mobile = phone_counts.pluck(0).sum()
total_work = phone_counts.pluck(1).sum()


print(f"Мобильные телефоны: {total_mobile.compute()}")
print(f"Рабочие/городские телефоны: {total_work.compute()}")

Мобильные телефоны: 7
Рабочие/городские телефоны: 6


## Лабораторная работа 12

In [1]:
import dask.bag as db
import json
import re
import pandas as pd

1. В файлах архиве `reviews_full.zip` находятся файлы, содержащие информацию об отзывах к рецептам в формате JSON Lines. Отзывы разделены на файлы в зависимости от оценки (например, в файле `reviews_1.json` находятся отзывы с оценкой 1). Считайте файлы из этого архива в виде `dask.bag`. Преобразуйте текстовое содержимое файлов в объекты python (с помощью модуля `json`). Выведите на экран первые 5 элементов полученного `bag`.

In [84]:
reviews_path = "12_dask_bag_data/reviews_full/*.json"

# Чтение всех JSON файлов с отзывами
# include_path=True позволяет получить путь к файлу для извлечения рейтинга
reviews_bag = db.read_text(reviews_path, include_path=True)
print(f"\nКоличество частей в bag: {reviews_bag.npartitions}")

# Функция для парсинга JSON
def parse_json(line_with_path):
    text, path = line_with_path
    review_data = json.loads(text)
    return review_data
    
# Преобразуем текстовые данные в объекты Python
parsed_reviews = reviews_bag.map(parse_json).filter(lambda x: x is not None)

first_5_reviews = parsed_reviews.take(5)
for i, review in enumerate(first_5_reviews, 1):
    print(f"\n{i}. Обзор:")
    for key, value in review.items():
        print(f"   {key}: {value}")
    print("-" * 50)

# Дополнительная информация о структуре данных
print(f"\nОбщее количество отзывов: {parsed_reviews.count().compute()}")


Количество частей в bag: 6

1. Обзор:
   user_id: 452355
   recipe_id: 292657
   date: 2016-05-08
   review: WOW!!! This is the best. I have never been able to make homemade enchiladas that taste like the Mexican restaurants. I made this last night for my family and they said they will never have enchiladas at the Mexican Restaurants again. Thanks for sharing.
--------------------------------------------------

2. Обзор:
   user_id: 329304
   recipe_id: 433404
   date: 2006-06-14
   review: This was good but the dressing needed something and I found it to be a little too sweet, next time I will experiment with some garlic and herbs and reduce the sugar slightly, thanks for sharing kcdlong!...Kitten
--------------------------------------------------

3. Обзор:
   user_id: 227932
   recipe_id: 2008187
   date: 1985-11-19
   review: Very good,it was a hit for my family. I used 6 cloves of garlic and had 1 lb beef and  Johnsonville sausage,1/2 lb hot and  1/2 lb honey garlic( which I want

2. Модифицируйте функцию разбора JSON таким образом, чтобы в каждый словарь c информацией об отзыве добавить ключ `rating`. Значение получите на основе названия файла (см. аргумент `include_path`), использовав для этого регулярное выражение.

In [88]:

def parse_json_with_rating(line_with_path):
    text, path = line_with_path
    
    review_data = json.loads(text)
    
    # Извлекаем рейтинг из названия файла
    filename = os.path.basename(path)
    # Ищем паттерн reviews_1.json, reviews_2.json и т.д.
    rating_match = re.search(r'reviews_(\d+)\.json', filename)
    if rating_match:
        review_data['rating'] = int(rating_match.group(1))
    else:
        review_data['rating'] = None
        
    return review_data
    

# Преобразуем с добавлением рейтинга
reviews_with_rating = db.read_text("12_dask_bag_data/reviews_full/*.json", include_path=True)
reviews_with_rating = reviews_with_rating.map(parse_json_with_rating).filter(lambda x: x is not None)


first_5_with_rating = reviews_with_rating.take(5)
for i, review in enumerate(first_5_with_rating, 1):
    print(f"\n{i}. ОТЗЫВ:")
    for key in ['rating', 'review', 'date']:  # Показываем только ключевые поля
        if key in review:
            value = review[key]
            if key == 'review' and value and len(str(value)) > 50:
                print(f"   {key}: {str(value)[:50]}...")
            else:
                print(f"   {key}: {value}")
    print("-" * 40)


print("\nРаспределение по рейтингам:")
ratings_count = reviews_with_rating.pluck('rating').frequencies()
ratings_result = ratings_count.compute()
for rating, count in sorted(ratings_result):
    print(f"  Рейтинг {rating}: {count} отзывов")


1. ОТЗЫВ:
   rating: 0
   review: WOW!!! This is the best. I have never been able to...
   date: 2016-05-08
----------------------------------------

2. ОТЗЫВ:
   rating: 0
   review: This was good but the dressing needed something an...
   date: 2006-06-14
----------------------------------------

3. ОТЗЫВ:
   rating: 0
   review: Very good,it was a hit for my family. I used 6 clo...
   date: 1985-11-19
----------------------------------------

4. ОТЗЫВ:
   rating: 0
   review: Made for ZWT-8 Family Picks after I saw these ment...
   date: 2019-05-21
----------------------------------------

5. ОТЗЫВ:
   rating: 0
   review: Very nice slaw. I especially like that it doesn't ...
   date: 1972-09-18
----------------------------------------

Распределение по рейтингам:
  Рейтинг 0: 487387 отзывов
  Рейтинг 1: 102679 отзывов
  Рейтинг 2: 112561 отзывов
  Рейтинг 3: 326792 отзывов
  Рейтинг 4: 1497732 отзывов
  Рейтинг 5: 6530389 отзывов


3. Посчитайте количество отзывов в исходном датасете.

In [91]:
reviews_with_rating.count().compute()

9057540

4. Отфильтруйте `bag`, сохранив только отзывы, оставленные в 2014 и 2015 годах.

In [93]:
def filter_2014_2015(review):
    if 'date' in review:
        date_str = str(review['date'])
        # Ищем год в дате (форматы могут быть разными: 2014-01-15, 15.01.2014 и т.д.)
        year_match = re.search(r'201[45]', date_str)
        return year_match is not None
    return False

filtered_reviews = reviews_with_rating.filter(filter_2014_2015)

print("Примеры отзывов за 2014-2015 годы:")
sample_filtered = filtered_reviews.take(3)
for i, review in enumerate(sample_filtered, 1):
    print(f"\n{i}. Дата: {review.get('date', 'N/A')}, Рейтинг: {review.get('rating', 'N/A')}")
    if 'review' in review:
        print(f"   Текст: {str(review['review'])[:100]}...")

# Подсчитываем количество
filtered_count = filtered_reviews.count()
print(f"\nКоличество отзывов за 2014-2015 годы: {filtered_count.compute():,}")

Примеры отзывов за 2014-2015 годы:

1. Дата: 2014-10-03, Рейтинг: 0
   Текст: Took this to a New Year&#039;s Eve Party. Everyone loved it! It&#039;s absolutely perfect, the flavo...

2. Дата: 2015-05-08, Рейтинг: 0
   Текст: Simple and easy way to enjoy a slice of pizza any time!  Well-toasted bread is the key - really toas...

3. Дата: 2015-06-30, Рейтинг: 0
   Текст: Delish!  I wanted to make this spicy so I used hot enchilada sauce and jalapeno refried beans.  I fo...

Количество отзывов за 2014-2015 годы: 735,274


5. Выполните препроцессинг отзывов:
    * привести строки к нижнему регистру
    * обрезать пробельные символы в начале и конце строки
    * удалите все символы, кроме английских букв и пробелов
    
Примените препроцессинг ко всем записям из `bag`, полученного в задании 4.

In [95]:
def preprocess_text(review):
    """
    Препроцессинг текста отзыва:
    - к нижнему регистру
    - обрезать пробелы
    - оставить только английские буквы и пробелы
    """
    if 'review' not in review or not review['review']:
        review['review_clean'] = ""
        return review
    
    text = str(review['review'])
    
    # 1. К нижнему регистру
    text = text.lower()
    
    # 2. Обрезать пробельные символы
    text = text.strip()
    
    # 3. Удалить все символы, кроме английских букв и пробелов
    text = re.sub(r'[^a-z\s]', '', text)
    
    # 4. Убрать множественные пробелы
    text = re.sub(r'\s+', ' ', text)
    
    review['review_clean'] = text
    return review

# Применяем препроцессинг к отфильтрованным отзывам
processed_reviews = filtered_reviews.map(preprocess_text)

# Показываем примеры до и после препроцессинга
print("Примеры препроцессинга:")
sample_processed = processed_reviews.take(3)
for i, review in enumerate(sample_processed, 1):
    print(f"\n{i}. Рейтинг: {review.get('rating', 'N/A')}")
    if 'review' in review:
        original = str(review['review'])[:80] + "..." if len(str(review['review'])) > 80 else str(review['review'])
        cleaned = review.get('review_clean', '')[:80] + "..." if len(review.get('review_clean', '')) > 80 else review.get('review_clean', '')
        print(f"   Оригинал: {original}")
        print(f"   Очищенный: {cleaned}")

Примеры препроцессинга:

1. Рейтинг: 0
   Оригинал: Took this to a New Year&#039;s Eve Party. Everyone loved it! It&#039;s absolutel...
   Очищенный: took this to a new years eve party everyone loved it its absolutely perfect the ...

2. Рейтинг: 0
   Оригинал: Simple and easy way to enjoy a slice of pizza any time!  Well-toasted bread is t...
   Очищенный: simple and easy way to enjoy a slice of pizza any time welltoasted bread is the ...

3. Рейтинг: 0
   Оригинал: Delish!  I wanted to make this spicy so I used hot enchilada sauce and jalapeno ...
   Очищенный: delish i wanted to make this spicy so i used hot enchilada sauce and jalapeno re...


6. Посчитайте количество отзывов в датасете, полученном в результате решения задачи 5. В случае ошибок прокомментируйте результат и исправьте функцию препроцессинга.

In [101]:

filtered_count_computed = filtered_reviews.count().compute()
processed_count = processed_reviews.count().compute()

print(f"Количество отзывов ДО препроцессинга: {filtered_count_computed}")
print(f"Количество отзывов ПОСЛЕ препроцессинга: {processed_count}")

# Проверим общую статистику по длине текстов
print(f"\nОбщая статистика по длине текстов:")
text_lengths = processed_reviews.map(lambda x: len(x.get('review_clean', '')))
avg_length = text_lengths.mean().compute()
max_length = text_lengths.max().compute()
min_length = text_lengths.min().compute()

print(f"  Средняя длина: {avg_length:.1f} символов")
print(f"  Минимальная длина: {min_length} символов")
print(f"  Максимальная длина: {max_length} символов")

# Проверим сколько отзывов стали пустыми после препроцессинга
empty_reviews_count = processed_reviews.filter(lambda x: len(x.get('review_clean', '')) == 0).count().compute()
print(f"  Пустых отзывов после обработки: {empty_reviews_count} ({empty_reviews_count/processed_count*100:.2f}%)")

Количество отзывов ДО препроцессинга: 735274
Количество отзывов ПОСЛЕ препроцессинга: 735274

Общая статистика по длине текстов:
  Средняя длина: 264.7 символов
  Минимальная длина: 0 символов
  Максимальная длина: 6533 символов
  Пустых отзывов после обработки: 47 (0.01%)


7. Посчитайте, как часто в наборе, полученном в задании 5, встречается та или иная оценка

In [103]:

# Подсчитываем частоту каждой оценки
rating_freq = processed_reviews.pluck('rating').frequencies()

# Вычисляем общее количество для процентного соотношения
total_processed = processed_reviews.count()

# Выводим результаты
rating_results = rating_freq.compute()
total = total_processed.compute()

print("Распределение оценок в отфильтрованных отзывах:")
print("-" * 50)
for rating, count in sorted(rating_results):
    percentage = (count / total) * 100
    print(f"Рейтинг {rating}: {count:,} отзывов ({percentage:.1f}%)")

print(f"\nВсего: {total:,} отзывов")

Распределение оценок в отфильтрованных отзывах:
--------------------------------------------------
Рейтинг 0: 42,472 отзывов (5.8%)
Рейтинг 1: 9,246 отзывов (1.3%)
Рейтинг 2: 9,380 отзывов (1.3%)
Рейтинг 3: 26,532 отзывов (3.6%)
Рейтинг 4: 119,413 отзывов (16.2%)
Рейтинг 5: 528,231 отзывов (71.8%)

Всего: 735,274 отзывов


8. Найдите среднее значение `rating` в выборке

In [107]:
# Вычисляем средний рейтинг
avg_rating = processed_reviews.pluck('rating').mean()

# Также посчитаем медиану и стандартное отклонение для полноты
rating_stats = processed_reviews.pluck('rating')
avg_result = avg_rating.compute()

print(f"Средний рейтинг отзывов за 2014-2015 годы: {avg_result:.2f}")

Средний рейтинг отзывов за 2014-2015 годы: 4.39


9. Используя метод `foldby`, подсчитать максимальную длину отзывов в зависимости от оценки `rating` в наборе, полученном в задании 5.

In [117]:
def get_review_length(review):
    """Возвращает кортеж (рейтинг, длина_текста)"""
    clean_text = review.get('review_clean', '')
    return (review['rating'], len(clean_text))

def max_reducer(acc, value):
    """Функция для нахождения максимума - исправленная версия"""
    # acc и value - это кортежи (рейтинг, длина)
    # Сравниваем только длины (второй элемент кортежа)
    return acc if acc[1] > value[1] else value

# Применяем foldby с правильным initial значением
max_lengths_by_rating = processed_reviews.map(get_review_length).foldby(
    key=lambda x: x[0],  # ключ - рейтинг (первый элемент кортежа)
    binop=max_reducer,   # операция - нахождение максимума по длине
    initial=(0, 0),      # начальное значение - кортеж (рейтинг, длина)
    combine=max_reducer  # функция комбинирования
)

max_lengths_result = max_lengths_by_rating.compute()
for rating_max_tuple in sorted(max_lengths_result):
    rating, max_length = rating_max_tuple
    print(f"Рейтинг {rating}: {max_length} символов")

Рейтинг 0: (0, 6533) символов
Рейтинг 1: (1, 2854) символов
Рейтинг 2: (2, 2809) символов
Рейтинг 3: (3, 3145) символов
Рейтинг 4: (4, 6533) символов
Рейтинг 5: (5, 5275) символов
